In [1]:
import numpy as np
import pandas as pd
from confluent_kafka import Producer
from connect import get_connection
from random import randint
from simulation import transaktion_factory
from time import sleep
import socket

from sqlalchemy import create_engine, inspect, text

from confluent_kafka import Consumer

In [2]:
from sqlalchemy import create_engine, inspect, text

USER = 'root'
PASSWORD = 'root'
HOST = 'localhost'          # see yml file
PORT = 3307
DATABASE = 'test_db_1'
 

def get_connection(database=DATABASE):
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}"\
            .format(USER, PASSWORD, HOST, PORT, database)
    )

engine = get_connection('mysql')
with engine.connect() as connection:
    insp = inspect(engine)
    db_list = insp.get_schema_names()
    print(f"Connection to the {HOST} for user {USER} created successfully.")
    if DATABASE not in db_list:
        sql = text(f"CREATE DATABASE {DATABASE} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
        result = connection.execute(sql)
        if result:
            print(f"Database {DATABASE} created!")

Connection to the localhost for user root created successfully.


In [3]:
query_1="""
DROP TABLE IF EXISTS M_Leihe.D_Buch;

CREATE TABLE M_Leihe.D_Buch
(
ID_Exemplar integer Primary Key not null,
Buch_ID integer, 
Genre varchar(50)
);
"""

query_2= """

INSERT INTO M_Leihe.D_Buch
SELECT 
 DISTINCT(ID_Exemplar), E.ID_Buch, B.Art  
FROM 
 test_db_1.Exemplare AS E
JOIN 
 test_db_1.Buch AS B
ON 
 E.ID_Buch = B.ID_Buch;
"""

In [4]:
from mysql.connector import connect

USER = 'root'
PASSWORD = 'root'
HOST = 'localhost'          # see yml file
PORT = 3307
#DATABASE='test_db_1'


connection = connect(user=USER, password=PASSWORD, host=HOST, port=PORT)#, database=DATABASE)

cursor = connection.cursor()

cursor.execute(query_1, multi=True)
cursor.close() 

connection = connect(user=USER, password=PASSWORD, host=HOST, port=PORT)
cursor = connection.cursor() 
cursor.execute(query_2, multi=True)

connection.commit()